In [367]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('RegexpTokenizer')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Error loading RegexpTokenizer: Package 'RegexpTokenizer'
[nltk_data]     not found in index
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk import RegexpTokenizer
from nltk.corpus import stopwords

def cleanText (text):
  tokenizer = RegexpTokenizer(r"\w+")                       # custom tokenizer to remove punctuation when tokenized
  stop_words = set(stopwords.words("english"))
  text = text.lower()                                       # making text all lower case
  text = tokenizer.tokenize(text)                          
  text = [word for word in text if not word in stop_words]  # removing stopwords

  return text

In [369]:
from google.colab import files

uploaded = files.upload()

Saving spamReviewData.csv to spamReviewData (1).csv


In [0]:
import io
import pandas as pd

data = pd.read_csv(io.BytesIO(uploaded["spamReviewData.csv"]))
data = data.rename(columns = {"text": "words"}).drop(columns="hotel")

In [0]:
# cleaning text
data["words"] = [cleanText(txt) for txt in data["words"]]

In [0]:
# creating total word count column
data["word_count"] = [len(txt) for txt in data["words"]]

In [0]:
from nltk.util import ngrams
data["word_bigram"] = [list(ngrams(txt, n=2)) for txt in data["words"]]

In [0]:
def getNGramCounts(nGrams):
  counts = [{} for ngram in nGrams]
  for i in range(len(nGrams)):
    for j in range(len(nGrams[i])):
      temp = nGrams[i][j]
      if temp in counts[i]:
        counts[i][temp] += 1
      else:
        counts[i][temp] = 1
  return counts

In [0]:
# creating unigram and bigram count dictionary for each review
data["word_unigram_counts"] = getNGramCounts(data["words"])
data["word_bigram_counts"] = getNGramCounts(data["word_bigram"])


In [0]:
# creating column for part-of-speech tags
from nltk import pos_tag
tagged_words = [pos_tag(txt) for txt in data["words"]]
data["tags"] = [[pair[1] for pair in review] for review in tagged_words]  

In [0]:
data["tag_bigram"] = [list(ngrams(txt, n=2)) for txt in data["tags"]]

In [0]:
# creating unigram and bigram count dictionary for each review
data["tag_unigram_counts"] = getNGramCounts(data["tags"])
data["tag_bigram_counts"] = getNGramCounts(data["tag_bigram"])

In [0]:
data = data.drop(columns=["word_bigram", "tag_bigram"])

In [399]:
data

,polarity,spam,words,word_count,word_unigram_counts,word_bigram_counts,tags,tag_unigram_counts,tag_bigram_counts
0,0,0,"[hard, rock, hotel, chicago, become, favorite,...",164,"{'hard': 1, 'rock': 3, 'hotel': 5, 'chicago': ...","{('hard', 'rock'): 1, ('rock', 'hotel'): 1, ('...","[JJ, NN, NN, VBZ, VBN, JJ, NN, VBD, JJS, CD, N...","{'JJ': 36, 'NN': 61, 'VBZ': 4, 'VBN': 4, 'VBD'...","{('JJ', 'NN'): 18, ('NN', 'NN'): 25, ('NN', 'V..."
1,0,0,"[wife, stay, downtown, event, attending, drake...",119,"{'wife': 1, 'stay': 3, 'downtown': 1, 'event':...","{('wife', 'stay'): 1, ('stay', 'downtown'): 1,...","[NN, VB, JJ, NN, VBG, JJ, NN, VBD, JJ, JJ, NN,...","{'NN': 43, 'VB': 7, 'JJ': 28, 'VBG': 4, 'VBD':...","{('NN', 'VB'): 1, ('VB', 'JJ'): 3, ('JJ', 'NN'..."
2,0,0,"[took, weekend, trip, wife, got, great, rate, ...",33,"{'took': 1, 'weekend': 1, 'trip': 1, 'wife': 1...","{('took', 'weekend'): 1, ('weekend', 'trip'): ...","[VBD, NN, NN, NN, VBD, JJ, NN, NN, VBD, JJ, NN...","{'VBD': 4, 'NN': 16, 'JJ': 8, 'NNS': 1, 'JJR':...","{('VBD', 'NN'): 1, ('NN', 'NN'): 6, ('NN', 'VB..."
3,0,0,"[stayed, great, reviews, true, highly, impress...",61,"{'stayed': 1, 'great': 1, 'reviews': 1, 'true'...","{('stayed', 'great'): 1, ('great', 'reviews'):...","[VBN, JJ, NNS, JJ, RB, JJ, JJ, NN, NN, NN, NN,...","{'VBN': 1, 'JJ': 9, 'NNS': 6, 'RB': 6, 'NN': 2...","{('VBN', 'JJ'): 1, ('JJ', 'NNS'): 2, ('NNS', '..."
4,0,0,"[returned, week, chicago, family, hotel, perfe...",53,"{'returned': 1, 'week': 1, 'chicago': 1, 'fami...","{('returned', 'week'): 1, ('week', 'chicago'):...","[VBN, NN, NNS, NN, NN, VBP, PRP, NN, JJ, NNS, ...","{'VBN': 3, 'NN': 17, 'NNS': 6, 'VBP': 4, 'PRP'...","{('VBN', 'NN'): 2, ('NN', 'NNS'): 3, ('NNS', '..."
...,...,...,...,...,...,...,...,...,...
1595,1,1,"[awful, experience, staff, rude, unhelpful, to...",29,"{'awful': 1, 'experience': 1, 'staff': 1, 'rud...","{('awful', 'experience'): 1, ('experience', 's...","[JJ, NN, NN, VBP, JJ, VBD, RB, NN, VBD, RB, JJ...","{'JJ': 6, 'NN': 8, 'VBP': 1, 'VBD': 5, 'RB': 6...","{('JJ', 'NN'): 3, ('NN', 'NN'): 2, ('NN', 'VBP..."
1596,1,0,"[experienced, bed, bugs, room, staying, omni, ...",34,"{'experienced': 1, 'bed': 1, 'bugs': 1, 'room'...","{('experienced', 'bed'): 1, ('bed', 'bugs'): 1...","[JJ, VBD, NNS, NN, VBG, JJ, JJ, NN, NN, NN, RB...","{'JJ': 7, 'VBD': 4, 'NNS': 3, 'NN': 14, 'VBG':...","{('JJ', 'VBD'): 1, ('VBD', 'NNS'): 1, ('NNS', ..."
1597,1,1,"[recently, stayed, affina, chicago, hotel, rea...",111,"{'recently': 1, 'stayed': 1, 'affina': 2, 'chi...","{('recently', 'stayed'): 1, ('stayed', 'affina...","[RB, VBN, RP, JJ, NN, RB, VBN, VBN, NN, NN, RB...","{'RB': 12, 'VBN': 9, 'RP': 1, 'JJ': 19, 'NN': ...","{('RB', 'VBN'): 3, ('VBN', 'RP'): 1, ('RP', 'J..."
1598,1,0,"[room, big, luxurious, befitting, price, tag, ...",51,"{'room': 2, 'big': 1, 'luxurious': 1, 'befitti...","{('room', 'big'): 1, ('big', 'luxurious'): 1, ...","[NN, JJ, JJ, NN, NN, NN, CD, NN, JJ, RB, VBD, ...","{'NN': 21, 'JJ': 9, 'CD': 3, 'RB': 4, 'VBD': 6...","{('NN', 'JJ'): 4, ('JJ', 'JJ'): 3, ('JJ', 'NN'..."


In [0]:
isReal = data["spam"] == 0
isFake = data["spam"] == 1
real_reviews = data[isReal]
fake_reviews = data[isFake]

In [0]:
def combineNGramCounts(dicts):
  ret = {}
  for dict in dicts:
    for key in dict.keys():
      if key in ret:
        ret[key] += dict[key]
      else:
        ret[key] = dict[key]
  return ret

In [0]:
real_reviews_word_unigram_counts = combineNGramCounts(real_reviews["word_unigram_counts"])
real_reviews_word_bigram_counts = combineNGramCounts(real_reviews["word_bigram_counts"])
real_reviews_tag_unigram_counts = combineNGramCounts(real_reviews["tag_unigram_counts"])
real_reviews_tag_bigram_counts = combineNGramCounts(real_reviews["tag_bigram_counts"])
fake_reviews_word_unigram_counts = combineNGramCounts(fake_reviews["word_unigram_counts"])
fake_reviews_word_bigram_counts = combineNGramCounts(fake_reviews["word_bigram_counts"])
fake_reviews_tag_unigram_counts = combineNGramCounts(fake_reviews["tag_unigram_counts"])
fake_reviews_tag_bigram_counts = combineNGramCounts(fake_reviews["tag_bigram_counts"])

In [0]:
real_reviews_word_unigram_counts = pd.DataFrame.from_dict(real_reviews_word_unigram_counts, orient= 'index', columns= ['count']).sort_values('count', ascending=False)
real_reviews_word_bigram_counts = pd.DataFrame.from_dict(real_reviews_word_bigram_counts, orient= 'index', columns= ['count']).sort_values('count', ascending=False)
real_reviews_tag_unigram_counts = pd.DataFrame.from_dict(real_reviews_tag_unigram_counts, orient= 'index', columns= ['count']).sort_values('count', ascending=False)
real_reviews_tag_bigram_counts = pd.DataFrame.from_dict(real_reviews_tag_bigram_counts, orient= 'index', columns= ['count']).sort_values('count', ascending=False)
fake_reviews_word_unigram_counts = pd.DataFrame.from_dict(fake_reviews_word_unigram_counts, orient= 'index', columns= ['count']).sort_values('count', ascending=False)
fake_reviews_word_bigram_counts = pd.DataFrame.from_dict(fake_reviews_word_bigram_counts, orient= 'index', columns= ['count']).sort_values('count', ascending=False)
fake_reviews_tag_unigram_counts = pd.DataFrame.from_dict(fake_reviews_tag_unigram_counts, orient= 'index', columns= ['count']).sort_values('count', ascending=False)
fake_reviews_tag_bigram_counts = pd.DataFrame.from_dict(fake_reviews_tag_bigram_counts, orient= 'index', columns= ['count']).sort_values('count', ascending=False)

In [0]:
# real_reviews_word_unigram_counts.nlargest(30, columns='count')
# fake_reviews_word_unigram_counts.nlargest(30, columns='count')
# real_reviews_tag_unigram_counts.nlargest(30, columns='count')
# fake_reviews_tag_unigram_counts.nlargest(30, columns='count')
# real_reviews_word_bigram_counts.nlargest(30, columns='count')
# fake_reviews_word_bigram_counts.nlargest(30, columns='count')
# real_reviews_tag_bigram_counts.nlargest(30, columns='count')
# fake_reviews_tag_bigram_counts.nlargest(30, columns='count')

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score
import numpy as np

In [406]:
X = [' '.join(words) for words in data["words"]]
Y = data['spam']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

clf = Pipeline([('vect', TfidfVectorizer(min_df= 3, sublinear_tf=True, norm='l2', ngram_range=(1, 3))),
                        ('chi',  SelectKBest(chi2, k=1200)),
                        ('clf', SVC())])

model = clf.fit(X_train, y_train)

ytest = np.array(y_test)

print("accuracy score:", accuracy_score(ytest, model.predict(X_test)))

accuracy score: 0.86
